In [1]:

import pandas as pd


Check ค่า null

In [2]:
data = pd.read_csv("retail_store_sales.csv")
data.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

Check ว่ามีสินค้า (Item) ใดบ้างที่มีราคาต่อหน่วย (Price Per Unit) แตกต่างกันหรือไม่

In [3]:
check=data.groupby("Item")["Price Per Unit"].nunique()
check[check>1]

Series([], Name: Price Per Unit, dtype: int64)

Check ราคาต่อหน่วย (Price Per Unit) และหมวดหมู่ (Category) ใดบ้างที่มีสินค้ามากกว่าหนึ่งชนิด (Item)

In [4]:
check=data.groupby(["Price Per Unit","Category"])["Item"].nunique()
check[check>1]

Series([], Name: Item, dtype: int64)

เติมคอลัม item
ของ 1 ชิ้นมี 1 ราคาในแต่ละ item
แสดงว่าเติมคอลัมด้วยวิธีนี้ได้

In [5]:
data[data["Price Per Unit"]==8]["Item"].unique()

array(['Item_3_BUT', 'Item_3_FOOD', 'Item_3_MILK', 'Item_3_BEV',
       'Item_3_FUR', 'Item_3_CEA', nan, 'Item_3_PAT', 'Item_3_EHE'],
      dtype=object)

เติม Price per unit จากจำนวนกับ total spent กับ Quantity

In [6]:
data[data["Price Per Unit"].isnull()].isnull().sum()

Transaction ID        0
Customer ID           0
Category              0
Item                609
Price Per Unit      609
Quantity              0
Total Spent           0
Payment Method        0
Location              0
Transaction Date      0
Discount Applied    205
dtype: int64

เติม Price per unit

In [17]:
data.loc[data['Price Per Unit'].isna(), 'Price Per Unit'] = data['Total Spent'] / data['Quantity']
data.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                 609
Price Per Unit         0
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [18]:
print(data)

      Transaction ID Customer ID       Category          Item  Price Per Unit  \
0        TXN_6867343     CUST_09     Patisserie   Item_10_PAT            18.5   
1        TXN_3731986     CUST_22  Milk Products  Item_17_MILK            29.0   
2        TXN_9303719     CUST_02       Butchers   Item_12_BUT            21.5   
3        TXN_9458126     CUST_06      Beverages   Item_16_BEV            27.5   
4        TXN_4575373     CUST_05           Food   Item_6_FOOD            12.5   
...              ...         ...            ...           ...             ...   
12570    TXN_9347481     CUST_18     Patisserie   Item_23_PAT            38.0   
12571    TXN_4009414     CUST_03      Beverages    Item_2_BEV             6.5   
12572    TXN_5306010     CUST_11       Butchers    Item_7_BUT            14.0   
12573    TXN_5167298     CUST_04      Furniture    Item_7_FUR            14.0   
12574    TXN_2407494     CUST_23           Food   Item_9_FOOD            17.0   

       Quantity  Total Spen

เติม Item จาก Price per unit ที่เติมแล้ว

In [19]:
# สร้าง mapping ของ Item ตาม Price Per U (เลือก Item แรกที่เจอในข้อมูล)
price_to_item = data.dropna(subset=['Item']).groupby('Price Per Unit')['Item'].first().to_dict()

# เติมค่าที่ขาดในคอลัมน์ Item โดยใช้ Price Per U
data['Item'] = data['Item'].fillna(data['Price Per Unit'].map(price_to_item))

data.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

ดูค่า mean กับ median

In [28]:
mean_quantity = data.groupby("Item")["Quantity"].mean()
median_quantity = data.groupby("Item")["Quantity"].median()

# แสดงผลลัพธ์
print("Mean Quantity per Item:\n", mean_quantity)
print("\nMedian Quantity per Item:\n", median_quantity)

Mean Quantity per Item:
 Item
Item_10_BEV     6.782609
Item_10_BUT     5.875000
Item_10_CEA     5.369863
Item_10_EHE     5.024390
Item_10_FOOD    5.107143
                  ...   
Item_9_EHE      5.915254
Item_9_FOOD     5.859155
Item_9_FUR      5.714286
Item_9_MILK     4.900000
Item_9_PAT      5.722222
Name: Quantity, Length: 200, dtype: float64

Median Quantity per Item:
 Item
Item_10_BEV     8.000000
Item_10_BUT     6.000000
Item_10_CEA     5.000000
Item_10_EHE     5.000000
Item_10_FOOD    5.000000
                  ...   
Item_9_EHE      7.000000
Item_9_FOOD     5.859155
Item_9_FUR      6.000000
Item_9_MILK     4.000000
Item_9_PAT      6.000000
Name: Quantity, Length: 200, dtype: float64


Check outlier

In [29]:
# คำนวณ IQR (Interquartile Range)
Q1 = data["Quantity"].quantile(0.25)  # ค่าของ Quartile ที่ 1 (25%)
Q3 = data["Quantity"].quantile(0.75)  # ค่าของ Quartile ที่ 3 (75%)
IQR = Q3 - Q1  # ช่วง Interquartile Range

# กำหนดช่วงที่ไม่มี Outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# ตรวจสอบว่ามีค่าไหนที่เป็น outlier บ้าง
outliers = data[(data["Quantity"] < lower_bound) | (data["Quantity"] > upper_bound)]

# แสดงผลลัพธ์
print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
print(f"พบ Outliers จำนวน {len(outliers)} รายการ:\n", outliers)

Q1: 3.0, Q3: 8.0, IQR: 5.0
Lower Bound: -4.5, Upper Bound: 15.5
พบ Outliers จำนวน 0 รายการ:
 Empty DataFrame
Columns: [Transaction ID, Customer ID, Category, Item, Price Per Unit, Quantity, Total Spent, Payment Method, Location, Transaction Date, Discount Applied]
Index: []


เติมค่า Quantity จาก Item โดยหาค่าเฉลี่ย mean ของแต่ละหมวดหมู่

In [23]:
data['Quantity'] = data.groupby('Item')['Quantity'].transform(lambda x: x.fillna(x.mean()))
data.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

เติมค่า Total spent จาก Price per unit * Quantity

In [32]:
# คำนวณ Total Spent
data["Total Spent"] = data["Price Per Unit"] * data["Quantity"]

print(data[["Price Per Unit", "Quantity", "Total Spent"]])

       Price Per Unit  Quantity  Total Spent
0                18.5      10.0        185.0
1                29.0       9.0        261.0
2                21.5       2.0         43.0
3                27.5       9.0        247.5
4                12.5       7.0         87.5
...               ...       ...          ...
12570            38.0       4.0        152.0
12571             6.5       9.0         58.5
12572            14.0      10.0        140.0
12573            14.0       6.0         84.0
12574            17.0       3.0         51.0

[12575 rows x 3 columns]


In [33]:
data.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

เติม discount

In [34]:
# เติมค่า missing ใน Discount Applied เป็น False
data["Discount Applied"] = data["Discount Applied"].fillna(False)

print(data[["Discount Applied"]])

       Discount Applied
0                  True
1                  True
2                 False
3                 False
4                 False
...                 ...
12570             False
12571             False
12572             False
12573              True
12574             False

[12575 rows x 1 columns]


In [35]:
data.isnull().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64